In [1]:
# Core Libraries
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  


In [2]:
df =pd.read_csv('response.csv')
df.head()

,Timestamp,What is your age?,What your gender?,What is your monthly income (UGX)?,Where do you reside?,How often do you shop on Jumia Uganda?,What is your average spending per order on Jumia (UGX)?,Which category do you shop from the most on Jumia Uganda?,What is your most frequently used payment method on Jumia?,Are you enrolled in Jumia Prime or any loyalty program?,How often do you repurchase from Jumia?,How satisfied are you with Jumia’s service?,How would you rate the availability of products you want on Jumia?,What is the primary reason for your purchase on Jumia?,What device do you primarily use to shop on Jumia?,What type of internet connection do you use for shopping?,Would you like to recommend Jumia to a friend or colleague?
0,10/10/2024 12:46:20,25-34,Male,"<450,000",Central,"Daily, A few times a year","<50,000",Electronics,"Mobile Money, Cash on Delivery",No,Rarely,3,2,"Convenience, Delivery speed","Smart phones, Laptops","Mobile data, Public Wi-Fi",Yes
1,10/10/2024 14:56:58,25-34,Male,None,Northern,A few times a year,"50,000-100,000",Electronics,Cash on Delivery,No,Rarely,5,3,Product variety,Smart phones,Home Wi-Fi,No
2,10/10/2024 15:01:38,25-34,Male,">2,000,000",Western,A few times a year,"50,000-100,000",Electronics,"Mobile Money, Cash on Delivery",No,Rarely,3,3,"Price, Convenience, Discounts/offers",Smart phones,Mobile data,Yes
3,10/10/2024 15:03:35,35-44,Male,"450,000-1,000,000",Central,A few times a year,"<50,000",Health & Beauty,Cash on Delivery,No,Rarely,1,3,Price,Smart phones,Mobile data,Yes
4,10/10/2024 15:04:12,25-34,Male,None,Northern,A few times a year,"50,000-100,000",Electronics,Mobile Money,No,Occassionally,3,3,Convenience,Smart phones,Mobile data,Yes


In [4]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Preprocessing
# Encode categorical variables
categorical_columns = [
    "What is your age?", "What your gender?", "Where do you reside?", 
    "How often do you shop on Jumia Uganda?", "Which category do you shop from the most on Jumia Uganda?", 
    "What is your most frequently used payment method on Jumia?", "Are you enrolled in Jumia Prime or any loyalty program?", 
    "How often do you repurchase from Jumia?", "What is the primary reason for your purchase on Jumia?", 
    "What device do you primarily use to shop on Jumia?", "What type of internet connection do you use for shopping?", 
    "Would you like to recommend Jumia to a friend or colleague?"
]

label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column].astype(str))  # Encode as strings to handle all categories
    label_encoders[column] = le

# Normalize numerical features
numerical_columns = [
    "What is your monthly income (UGX)?", "What is your average spending per order on Jumia (UGX)?", 
    "How satisfied are you with Jumia’s service?", "How would you rate the availability of products you want on Jumia?"
]

# Convert income and spending to numerical ranges (example: "<450,000" to 225000, "450,000-1,000,000" to 725000, etc.)
def income_to_numeric(income):
    if income == "None":
        return 0
    elif income == "<450,000":
        return 225000
    elif income == "450,000-1,000,000":
        return 725000
    elif income == ">2,000,000":
        return 2000000
    else:
        return 0  # Handle unknown values

def spending_to_numeric(spending):
    if spending == "<50,000":
        return 25000
    elif spending == "50,000-100,000":
        return 75000
    else:
        return 0  # Handle unknown values

df["What is your monthly income (UGX)?"] = df["What is your monthly income (UGX)?"].apply(income_to_numeric)
df["What is your average spending per order on Jumia (UGX)?"] = df["What is your average spending per order on Jumia (UGX)?"].apply(spending_to_numeric)

# Normalize numerical columns
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Drop timestamp (if not needed)
df = df.drop(columns=["imestamp"])

# Apply K-Means
kmeans = KMeans(n_clusters=3, random_state=42)  # Choose k=3 for example
df["cluster"] = kmeans.fit_predict(df.drop(columns=["cluster"]))  # Add cluster labels to the dataframe

# Visualize clusters using PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df.drop(columns=["cluster"]))
df["pca1"] = pca_result[:, 0]
df["pca2"] = pca_result[:, 1]

plt.scatter(df["pca1"], df["pca2"], c=df["cluster"], cmap="viridis")
plt.title("K-Means Clustering (k=3)")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.colorbar(label="Cluster")
plt.show()

# Inspect clusters
print(df.groupby("cluster").mean())  # Analyze cluster characteristics

KeyError: 'What is your age?'